In [1]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import pickle
import MDAnalysis, sys
from MDAnalysis import transformations
%config InlineBackend.figure_format = 'retina'
dt=0.5# fs


In [ ]:
ice='IceIh'

In [2]:
def extract_form_log(fn,dt):
    lines=open(fn,'r').readlines()
    start=np.where([('Step' in l) for l in lines])[-1][0]
    data={keyi.lower(): []  for keyi in lines[start].split()}
    for l in lines[start+1:]:
        ls=l.split()
        if ls[0].isdigit():
            for i, key in enumerate(data.keys()):
                data[key].append(float(ls[i]))
        else:
            break
    
    for key in data.keys():
        data[key]=np.array(data[key])
    data['time_ps']=dt*data['step']/1000
    return data

In [4]:
sims=dict()
for f in glob.glob('*_*/log.lammps'):
    try:
        sims[f.replace('/log.lammps','')]=extract_form_log(f,dt)
    except:
        pass

with open('1-bulk.pickle', 'wb') as handle:
    pickle.dump(sims, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('1-bulk.pickle', 'rb') as handle:
    sims = pickle.load(handle)

In [5]:
print(sims.keys())

dict_keys(['IceIh_1atm_270K', 'IceIh_1000atm_260K', 'IceIh_1000atm_270K', 'IceIh_1atm_260K', 'IceIh_2000atm_245K', 'IceIh_2000atm_255K', 'IceIh_1atm_275K', 'IceIh_2000atm_260K', 'IceIh_1atm_280K', 'IceIh_2000atm_240K', 'IceIh_1000atm_255K', 'IceIh_2000atm_250K'])


In [35]:
data=[]
for key in sims.keys():
    ls=key.split('_')
    p=round(float(ls[1].strip('atm')),0)
    t=round(float(ls[2].strip('K')),0)
    
    last=len(sims[key]['step'])//10
    cella=round(sims[key]['cella'][last:].mean(),3)
    cellb=round(sims[key]['cellb'][last:].mean(),3)
    cellc=round(sims[key]['cellc'][last:].mean(),3)
    cellalpha=round(sims[key]['cellalpha'][last:].mean(),3)
    cellbeta=round(sims[key]['cellbeta'][last:].mean(),3)
    cellgamma=round(sims[key]['cellgamma'][last:].mean(),3)
    
    dat=[p,t,cella,cellb,cellc,cellalpha,cellbeta,cellgamma]
    data.append(dat)

with open('avg_ice_box.dat','w') as fp:
    for dat in data:
        fp.write('\t'.join(['{}'.format(d) for d in dat])+'\n')
!cat avg_ice_box.dat

1.0	270.0	18.108	23.479	22.206	90.0	90.0	90.0
1000.0	260.0	18.032	23.381	22.114	90.0	90.0	90.0
1000.0	270.0	18.036	23.396	22.121	90.0	90.0	90.0
1.0	260.0	18.095	23.466	22.188	90.0	90.0	90.0
2000.0	245.0	17.952	23.271	22.018	90.0	90.0	90.0
2000.0	255.0	17.961	23.289	22.03	90.0	90.0	90.0
1.0	275.0	18.114	23.488	22.207	90.0	90.0	90.0
2000.0	260.0	17.964	23.288	22.033	90.0	90.0	90.0
1.0	280.0	18.12	23.494	22.219	90.0	90.0	90.0
2000.0	240.0	17.949	23.272	22.017	90.0	90.0	90.0
1000.0	255.0	18.025	23.372	22.11	90.0	90.0	90.0
2000.0	250.0	17.956	23.276	22.022	90.0	90.0	90.0
